# Finding a suitable location for opening a restaurant

## Problem statement
        A restaurants success and failure depends on variety of factors. One of those many factors is its location. The location a restaurant is as important as its quality and taste, besides location of a restaurant is an one-off factor. So it's important that we find a optimal location to run a restaurant. As a data scientist, we are required to properly analyze the location of various neighborhoods in the city and find a proper location to open a restaurant.
        This problem is mostly faced by all the entrepreneurs thriving to open up a business. In this project we are gonna focus on the restaurant location.

## Data sets to be used:
 For the given problem I may use the following data sets,
   - Canada neighborhood data
                            
   - Latitude and Longitude data for the corresponding neighborhood
                            
   - Venue Data

## Description on dataset
        First, the Canada neighborhood data is available in this wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. This dataset contains the Borough and corresponding neighborhood data. We will be using a Web scraping tool called BeautifulSoup to scrape the website and develop a data frame. The latitude and longitude dataset is already provided by the IBM team. This dataset contains all the latitude and longitude of all the neighborhoods. So we can match the neighborhood in both the dataframes and create a new dataframe. For the venue data, we will be using the foursquare api. I have already created a developer account in the website which gives us access to its venue data. This API will be used to extract all the restaurants in a particular neighborhood.

## Step1: Data Cleaning and Visualization
In this step, I have imported all the required libraries and imported the dataset which was prepared in another notebook. So the code to prepare the dataset is not mentioned here.
Further, in this step we will be visualizing the data using folium library. 

In [30]:
import pandas as pd
import folium
import numpy as np
import requests

In [31]:
df = pd.read_csv('canada.csv')
df = df.drop(['Unnamed: 0', 'Postal Code.1'], axis=1)
df.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [32]:
latitude = 43.6534817
longitude = -79.3839347

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto.save('visual_neighborhood.jpeg')
display(map_toronto)

## Step2: Fetching Indian restaurants in every neighborhood 
In this step we will be using FourSquare API to fetch all the Indian cuisine restaurants from each neighborhood. Its returns the value as a json file. So I have extracted only important elements from the json and is stored in 'res_list'. From this list a dataframe is created which contains elements like borough, Neighborhood name, latitude, longitude and restaurants name, latitude, longitude. we will be using ths data frame for further clustering and analysis. This data frame is called as new_df.
After creating the database we will be visualizing the restaurants in folium map by plotting their lat and long coordinates. This is done to get a better idea on how close the restaurants are located to each other. 

In [33]:
# FourSquare credentials
CLIENT_ID = 'GJEDABXLVSBFAFLOMJP4WIPJG12OW0VRDJP4NFFZXWANAENL' # your Foursquare ID
CLIENT_SECRET = 'MK1YEKAAWAHN1FH1TM0VMWOCGMAW1XTYXNV5FIDMN2DB0F0T' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GJEDABXLVSBFAFLOMJP4WIPJG12OW0VRDJP4NFFZXWANAENL
CLIENT_SECRET:MK1YEKAAWAHN1FH1TM0VMWOCGMAW1XTYXNV5FIDMN2DB0F0T


In [34]:
search_query = 'indian restaurant'
radius = 500
LIMIT = 100

In [35]:
res_list = []
for borough, n, lat, lng in zip(df['Borough'], df['Neighborhood'], df['Latitude'], df['Longitude']):
        latlong = df[df['Neighborhood'] == n]
        lat = latlong['Latitude'].iloc[0]
        long = latlong['Longitude'].iloc[0]
        url = f'https://api.foursquare.com/v2/venues/search?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={lat},{long}&v={VERSION}&query={search_query}&radius={radius}&limit={LIMIT}'
        results = requests.get(url).json()
        results = results['response']['venues']
        
        for v in results:
            res_name = v['name']
            res_lat = v['location']['lat']
            res_lng = v['location']['lng']
            res_list.append([borough, n, lat, lng, res_name, res_lat, res_lng])

In [36]:
res_list[0:3]

[['Scarborough',
  'Guildwood, Morningside, West Hill',
  43.7635726,
  -79.1887115,
  'Wonder Season Chinese Restaurant',
  43.76535407,
  -79.19053556],
 ['Scarborough',
  'Cedarbrae',
  43.773136,
  -79.23947609999998,
  "Terry's Restaurant",
  43.77470245986757,
  -79.24117495234971],
 ['Scarborough',
  'Cedarbrae',
  43.773136,
  -79.23947609999998,
  "Terry's Restaurant & Bar",
  43.77477972560302,
  -79.241042790795]]

In [37]:
new_df = pd.DataFrame(res_list, columns=['Borough', 'Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Name of Restaurant', 'Restaurant Latitude', 'Restaurant Longitude'])

In [38]:
new_df

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Name of Restaurant,Restaurant Latitude,Restaurant Longitude
0,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Wonder Season Chinese Restaurant,43.765354,-79.190536
1,Scarborough,Cedarbrae,43.773136,-79.239476,Terry's Restaurant,43.774702,-79.241175
2,Scarborough,Cedarbrae,43.773136,-79.239476,Terry's Restaurant & Bar,43.774780,-79.241043
3,Scarborough,Cedarbrae,43.773136,-79.239476,terry's restaurant,43.774969,-79.240872
4,Scarborough,Cedarbrae,43.773136,-79.239476,Thai One On,43.774468,-79.241268
...,...,...,...,...,...,...,...
617,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056
618,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,Il Paesano Pizzeria & Restaurant,43.601280,-79.545028
619,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,ASA Korean Restaurant,43.603950,-79.537700
620,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,McDonald's,43.630002,-79.518198


In [39]:
latitude = 43.6534817
longitude = -79.3839347

res_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, name in zip(new_df['Restaurant Latitude'], new_df['Restaurant Longitude'], new_df['Neighborhood'], new_df['Name of Restaurant']):
    label = '{}, {}'.format(name, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(res_toronto)  
    
res_toronto.save('visual_restaurants.html')
display(res_toronto)

## Step 3: Clustering the Data
Clustering the restaurants using KMeans clustering. Clustering is done to help us find which part of Toronto has most no. of restaurants. We can use this insight to find where there will be more demand for a particular restaurant.
The clustered data will be plotted by 4 differents colours on toronto map. 

In [40]:
from sklearn.cluster import KMeans

In [41]:
kclusters = 4

restaurant_clustering = new_df.drop(['Borough', 'Neighborhood Latitude','Neighborhood Longitude', 'Neighborhood', 'Name of Restaurant'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0)
kmeans.fit(restaurant_clustering)

value = kmeans.labels_[0:]
print(value)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [42]:
new_df.insert(0, 'Cluster Labels', value)

In [43]:
new_df.head(2)

,Cluster Labels,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Name of Restaurant,Restaurant Latitude,Restaurant Longitude
0,2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Wonder Season Chinese Restaurant,43.765354,-79.190536
1,2,Scarborough,Cedarbrae,43.773136,-79.239476,Terry's Restaurant,43.774702,-79.241175


In [44]:
new_df.groupby('Cluster Labels').count()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Name of Restaurant,Restaurant Latitude,Restaurant Longitude
Cluster Labels,,,,,,,
0,529,529,529,529,529,529,529
1,39,39,39,39,39,39,39
2,26,26,26,26,26,26,26
3,28,28,28,28,28,28,28


In [45]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [46]:
latitude = 43.6534817
longitude = -79.3839347

clustered_restaurant_map = folium.Map(location=[latitude, longitude], zoom_start=10)


rainbow = ['#8000ff', '#2adddd', '#FFA500', '#ff0000'] 

markers_colors = []
for lat, lng, neighborhood, name, cluster in zip(new_df['Restaurant Latitude'], new_df['Restaurant Longitude'], new_df['Neighborhood'], new_df['Name of Restaurant'], new_df['Cluster Labels']):
    label = 'Restaurant Name - {}, \nNeighborhood - {}, \ncluster - {}'.format(name, neighborhood, cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7,
        parse_html=False).add_to(clustered_restaurant_map)  

clustered_restaurant_map.save('cluster_visual_restaurants.html')
display(clustered_restaurant_map)


## Step 4 Examine the Cluster
Examining the clusters. From each cluster we can find the most no. of restaurants in a particular neighborhood.

In [47]:
cluster0 = new_df.loc[new_df['Cluster Labels'] == 0]
cluster0

,Cluster Labels,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Name of Restaurant,Restaurant Latitude,Restaurant Longitude
39,0,East Toronto,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167
42,0,East York,Thorncliffe Park,43.705369,-79.349372,Hamdi restaurant,43.704601,-79.349699
43,0,East York,Thorncliffe Park,43.705369,-79.349372,Trupti Indian Grocery,43.707003,-79.350447
44,0,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,Sunshine Caribbean Restaurant,43.680767,-79.338146
45,0,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,Makkah Restaurant,43.680642,-79.336613
...,...,...,...,...,...,...,...,...
610,0,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Subway,43.667527,-79.388919
611,0,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,A&W,43.663094,-79.383971
612,0,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Subway,43.659006,-79.384760
613,0,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Mullins Irish Alehouse,43.666303,-79.387936


In [48]:
cluster1 = new_df.loc[new_df['Cluster Labels'] == 1]
cluster1

,Cluster Labels,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Name of Restaurant,Restaurant Latitude,Restaurant Longitude
25,1,North York,"Willowdale, Willowdale East",43.770120,-79.408493,Symposium Cafe Restaurant & Lounge,43.771075,-79.413396
26,1,North York,"Willowdale, Willowdale East",43.770120,-79.408493,Trio Restaurant,43.769058,-79.414377
27,1,North York,"Willowdale, Willowdale East",43.770120,-79.408493,Restaurant At Novotel North York Toronto,43.769609,-79.414167
28,1,North York,"Willowdale, Willowdale East",43.770120,-79.408493,Hooraiya Sushi 자매 족발 & 보쌈,43.766683,-79.412261
29,1,North York,"Willowdale, Willowdale East",43.770120,-79.408493,Chattanooga Restaurant,43.773981,-79.413992
30,1,North York,"Willowdale, Willowdale East",43.770120,-79.408493,Ichiban Fish House,43.765995,-79.410813
31,1,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,Orly Restaurant & Grill,43.754493,-79.443507
32,1,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,Subway,43.755741,-79.440502
33,1,North York,"Northwood Park, York University",43.767980,-79.487262,Antepli. Turkish cafe and restaurant,43.765614,-79.488673
34,1,North York,"Northwood Park, York University",43.767980,-79.487262,J's Bar & Restaurant,43.765461,-79.489437


In [49]:
cluster2 = new_df.loc[new_df['Cluster Labels'] == 2]
cluster2

,Cluster Labels,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Name of Restaurant,Restaurant Latitude,Restaurant Longitude
0,2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Wonder Season Chinese Restaurant,43.765354,-79.190536
1,2,Scarborough,Cedarbrae,43.773136,-79.239476,Terry's Restaurant,43.774702,-79.241175
2,2,Scarborough,Cedarbrae,43.773136,-79.239476,Terry's Restaurant & Bar,43.774780,-79.241043
3,2,Scarborough,Cedarbrae,43.773136,-79.239476,terry's restaurant,43.774969,-79.240872
4,2,Scarborough,Cedarbrae,43.773136,-79.239476,Thai One On,43.774468,-79.241268
5,2,Scarborough,Cedarbrae,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142
6,2,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304,Karaikudi Chettinad South Indian Restaurant,43.756042,-79.276276
7,2,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304,Kim Kim restaurant,43.753833,-79.276611
8,2,Scarborough,"Wexford, Maryvale",43.750071,-79.295849,Wexford Restaurant,43.746030,-79.293843
9,2,Scarborough,"Wexford, Maryvale",43.750071,-79.295849,Colony restaurant,43.745477,-79.296889


In [50]:
cluster3 = new_df.loc[new_df['Cluster Labels'] == 3]
cluster3

,Cluster Labels,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Name of Restaurant,Restaurant Latitude,Restaurant Longitude
572,3,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,Morningstar Restaurant,43.669468,-79.442361
586,3,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,Oriental City Restaurant,43.690984,-79.471979
587,3,York,"Runnymede, The Junction North",43.673185,-79.487262,Bigger's Jamaican Restaurant,43.672725,-79.482508
588,3,West Toronto,"High Park, The Junction South",43.661608,-79.464763,Indian Road Crescent Public School,43.662491,-79.459608
589,3,West Toronto,"High Park, The Junction South",43.661608,-79.464763,Toronto Indian Road YMCA Before and After Scho...,43.662718,-79.460031
590,3,West Toronto,"High Park, The Junction South",43.661608,-79.464763,Indian Grove,43.657400,-79.459998
591,3,West Toronto,"High Park, The Junction South",43.661608,-79.464763,503 Indian Grove,43.664610,-79.462226
592,3,West Toronto,"High Park, The Junction South",43.661608,-79.464763,Leno's Bar & Restaurant,43.665349,-79.464447
593,3,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,Indian road,43.652362,-79.455718
594,3,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,Domani Restaurant & Wine Bar,43.649235,-79.450229


## Step5: Grouping
Grouping by neighborhood and borough to find the no. of restaurants in each. 

In [51]:
cluster1.groupby(['Borough', 'Neighborhood']).count()

Cluster Labels  \
Borough         Neighborhood                                                      
Central Toronto Davisville                                                    7   
                Davisville North                                              4   
                Forest Hill North & West, Forest Hill Road Park               2   
                North Toronto West,  Lawrence Park                            2   
East York       Leaside                                                       2   
North York      Bathurst Manor, Wilson Heights, Downsview North               2   
                Bedford Park, Lawrence Manor East                             5   
                Glencairn                                                     2   
                Lawrence Manor, Lawrence Heights                              1   
                Northwood Park, York University                               5   
                Willowdale, Willowdale East                                   6   
York            Caledonia-Fairbanks                                           1   

                                                                 Neighborhood Latitude  \
Borough         Neighborhood                                                             
Central Toronto Davisville                                                           7   
                Davisville North                                                     4   
                Forest Hill North & West, Forest Hill Road Park                      2   
                North Toronto West,  Lawrence Park                                   2   
East York       Leaside                                                              2   
North York      Bathurst Manor, Wilson Heights, Downsview North                      2   
                Bedford Park, Lawrence Manor East                                    5   
                Glencairn                                                            2   
                Lawrence Manor, Lawrence Heights                                     1   
                Northwood Park, York University                                      5   
                Willowdale, Willowdale East                                          6   
York            Caledonia-Fairbanks                                                  1   

                                                                 Neighborhood Longitude  \
Borough         Neighborhood                                                              
Central Toronto Davisville                                                            7   
                Davisville North                                                      4   
                Forest Hill North & West, Forest Hill Road Park                       2   
                North Toronto West,  Lawrence Park                                    2   
East York       Leaside                                                               2   
North York      Bathurst Manor, Wilson Heights, Downsview North                       2   
                Bedford Park, Lawrence Manor East                                     5   
                Glencairn                                                             2   
                Lawrence Manor, Lawrence Heights                                      1   
                Northwood Park, York University                                       5   
                Willowdale, Willowdale East                                           6   
York            Caledonia-Fairbanks                                                   1   

                                                                 Name of Restaurant  \
Borough         Neighborhood                                                          
Central Toronto Davisville                                                        7   
                Davisville North                                                  4   
                Fo

#### Recommended location from cluster 2:
The below neighborhoods are recomended for restaturants,
- Steeles West, L'Amoreaux West
- Cedarbrae


In [52]:
cluster2.groupby(['Borough', 'Neighborhood']).count()

Cluster Labels  \
Borough     Neighborhood                                                         
East York   Parkview Hill, Woodbine Gardens                                  1   
Scarborough Agincourt                                                        4   
            Cedarbrae                                                        5   
            Clarks Corners, Tam O'Shanter, Sullivan                          3   
            Dorset Park, Wexford Heights, Scarborough Town ...               2   
            Guildwood, Morningside, West Hill                                1   
            Milliken, Agincourt North, Steeles East, L'Amor...               2   
            Steeles West, L'Amoreaux West                                    5   
            Wexford, Maryvale                                                3   

                                                                Neighborhood Latitude  \
Borough     Neighborhood                                                                
East York   Parkview Hill, Woodbine Gardens                                         1   
Scarborough Agincourt                                                               4   
            Cedarbrae                                                               5   
            Clarks Corners, Tam O'Shanter, Sullivan                                 3   
            Dorset Park, Wexford Heights, Scarborough Town ...                      2   
            Guildwood, Morningside, West Hill                                       1   
            Milliken, Agincourt North, Steeles East, L'Amor...                      2   
            Steeles West, L'Amoreaux West                                           5   
            Wexford, Maryvale                                                       3   

                                                                Neighborhood Longitude  \
Borough     Neighborhood                                                                 
East York   Parkview Hill, Woodbine Gardens                                          1   
Scarborough Agincourt                                                                4   
            Cedarbrae                                                                5   
            Clarks Corners, Tam O'Shanter, Sullivan                                  3   
            Dorset Park, Wexford Heights, Scarborough Town ...                       2   
            Guildwood, Morningside, West Hill                                        1   
            Milliken, Agincourt North, Steeles East, L'Amor...                       2   
            Steeles West, L'Amoreaux West                                            5   
            Wexford, Maryvale                                                        3   

                                                                Name of Restaurant  \
Borough     Neighborhood                                                             
East York   Parkview Hill, Woodbine Gardens                                      1   
Scarborough Agincourt                                                            4   
            Cedarbrae                                                            5   
            Clarks Corners, Tam O'Shanter, Sullivan                              3   
            Dorset Park, Wexford Heights, Scarborough Town ...                   2   
            Guildwood, Morningside, West Hill                                    1   
            Milliken, Agincourt North, Steeles East, L'Amor...                   2   
            Steeles West, L'Amoreaux West                                        5   
            Wexford, Maryvale                                                    3   

                                                                Restaurant Latitude  \
Borough     Neighborhood                                                              
East York   Parkview Hill, Woodbine Gardens                          

In [53]:
cluster3.groupby(['Borough', 'Neighborhood']).count()

Cluster Labels  \
Borough      Neighborhood                                                         
Etobicoke    Alderwood, Long Branch                                           2   
             Mimico NW, The Queensway West, South of Bloor, ...               1   
             New Toronto, Mimico South, Humber Bay Shores                     1   
Mississauga  Canada Post Gateway Processing Centre                            2   
North York   Humber Summit                                                    1   
West Toronto Dufferin, Dovercourt Village                                     1   
             High Park, The Junction South                                    5   
             Parkdale, Roncesvalles                                           5   
             Runnymede, Swansea                                               8   
York         Del Ray, Mount Dennis, Keelsdale and Silverthorn                 1   
             Runnymede, The Junction North                                    1   

                                                                 Neighborhood Latitude  \
Borough      Neighborhood                                                                
Etobicoke    Alderwood, Long Branch                                                  2   
             Mimico NW, The Queensway West, South of Bloor, ...                      1   
             New Toronto, Mimico South, Humber Bay Shores                            1   
Mississauga  Canada Post Gateway Processing Centre                                   2   
North York   Humber Summit                                                           1   
West Toronto Dufferin, Dovercourt Village                                            1   
             High Park, The Junction South                                           5   
             Parkdale, Roncesvalles                                                  5   
             Runnymede, Swansea                                                      8   
York         Del Ray, Mount Dennis, Keelsdale and Silverthorn                        1   
             Runnymede, The Junction North                                           1   

                                                                 Neighborhood Longitude  \
Borough      Neighborhood                                                                 
Etobicoke    Alderwood, Long Branch                                                   2   
             Mimico NW, The Queensway West, South of Bloor, ...                       1   
             New Toronto, Mimico South, Humber Bay Shores                             1   
Mississauga  Canada Post Gateway Processing Centre                                    2   
North York   Humber Summit                                                            1   
West Toronto Dufferin, Dovercourt Village                                             1   
             High Park, The Junction South                                            5   
             Parkdale, Roncesvalles                                                   5   
             Runnymede, Swansea                                                       8   
York         Del Ray, Mount Dennis, Keelsdale and Silverthorn                         1   
             Runnymede, The Junction North                                            1   

                                                                 Name of Restaurant  \
Borough      Neighborhood                                                             
Etobicoke    Alderwood, Long Branch                                               2   
             Mimico NW, The Queensway West, South of Bloor, ...                   1   
             New Toronto, Mimico South, Humber Bay Shores                         1   
Mississauga  Canada Post Gateway Processing Centre                                2   
North York   Humber Summit                                                        1   
West Toronto Dufferin

## Result
    From the analysis we conclude that the neighborhoods listed in cluster0 is not an ideal place to open a new Indian restaurant as the location is already packed with more no. of Indian restaurants. So neighborhood from other three clusters are recommended. To be more accurate and specific the following neighborhood is recommended from each cluster.

#### Recommended location from cluster 2:
The below neighborhoods are recomended for restaturants,
- Steeles West, L'Amoreaux West
- Cedarbrae

#### Recommended location from cluster 3:
The below neighborhoods are recomended for restaturants,
- Runnymede, Swansea
- Parkdale, Roncesvalles
- High Park, The Junction South

#### Recommended location from cluster 4:
The below neighborhoods are recomended for restaturants,
- Davisville
- Willowdale, Willowdale East






## Conclusion
    Purpose of this project was to find an good location to open an Indian restaurant. From research we found that an ideal place to open a restaurant is where there is average no. of competitors. Based on this assumption the entire project as been completed and recommended neighborhoods have been produced.
    For further analysis, we can also include data like, Indian population, crime rate, labor wage etc. 